In [1]:
!pip install "unstructured[pdf]"
!pip install unstructured-inference

  Using cached google_cloud_vision-3.7.4-py2.py3-none-any.whl.metadata (5.2 kB)
  Using cached effdet-0.4.1-py3-none-any.whl.metadata (33 kB)
  Using cached pycocotools-2.0.8-cp312-cp312-win_amd64.whl.metadata (1.1 kB)
  Using cached omegaconf-2.3.0-py3-none-any.whl.metadata (3.9 kB)
  Using cached Deprecated-1.2.14-py2.py3-none-any.whl.metadata (5.4 kB)
  Using cached rsa-4.9-py3-none-any.whl.metadata (4.2 kB)
Using cached effdet-0.4.1-py3-none-any.whl (112 kB)
Using cached google_cloud_vision-3.7.4-py2.py3-none-any.whl (467 kB)
   ---------------------------------------- 0.0/3.5 MB ? eta -:--:--
   - -------------------------------------- 0.1/3.5 MB 3.3 MB/s eta 0:00:02
   ---- ----------------------------------- 0.4/3.5 MB 6.2 MB/s eta 0:00:01
   ------ --------------------------------- 0.6/3.5 MB 6.3 MB/s eta 0:00:01
   ---------- ----------------------------- 0.9/3.5 MB 6.6 MB/s eta 0:00:01
   -------------- ------------------------- 1.3/3.5 MB 6.2 MB/s eta 0:00:01
   ------------


[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
from unstructured.partition.pdf import partition_pdf

# Returns a List[Element] present in the pages of the parsed pdf document
elements = partition_pdf(filename="whatiscs.pdf",
                        strategy="auto",
                        infer_table_structure=True,
                        chunking_strategy="by_title",
                        max_characters=15000,
                        combine_text_under_n_chars=13000
                        )

d:\multivector+opensource_llm\venv\Lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Some weights of the model checkpoint at microsoft/table-transformer-structure-recognition were not used when initializing TableTransformerForObjectDetection: ['model.backbone.conv_encoder.model.layer2.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer3.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer4.0.downsample.1.num_batches_tracked']
- This IS expected if you are initializing TableTransformerForObjectDetection from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TableTransformerForObjectDetection from the ch

In [3]:
tables = [el.text for el in elements if el.category == "Table"]
texts=[el.text for el in elements if el.category == "CompositeElement"]

In [4]:
texts

['WHAT IS COMPUTER SCIENCE ALL ABOUT?\n\nH. Conrad Cunningham and Pallavi Tadepalli Department of Computer and Information Science University of Mississippi\n\nCOMPUTERS EVERYWHERE As scientific and engineering disciplines go, computer science is still quite young. Although the mathematical roots of computer science go back more than a thousand years, it is only with the invention of the programmable electronic digital computer during the World War II era of the 1930s and 1940s that the modern discipline of computer science began to take shape. As it has developed, computer science includes theoretical studies, experimental methods, and engineering design all in one discipline.\n\nOne of the first computers was the ENIAC (Electronic Numerical Integrator and Computer), developed in the mid-1940s at the University of Pennsylvania. When construction was completed in 1946, the ENIAC cost about $500,000. In today’s terms, that is about $5,000,000. It weighed 30 tons, occupied as much space 

In [6]:
tables

['High-level language program Compiler Machine language program Computer',
 'Job Title Responsibilities • Discuss system requirements with users • Define system to satisfy user needs; specify hardware and Systems Analyst software requirements for system • Possess good communication skills • Have extensive knowledge of both technical capabilities and business organization • Design and build software using existing models • Analyze user’s needs; design, implement, test, and Software Engineer maintain large software systems • Apply software practices to ensure product reliability, cost effectiveness, and security • Responsible for entire "life cycle" of a software system • Design, write, test, and maintain programs for a variety of Programmer applications • Design and implement new test cases to test product functionality Software Tester • Assess functionality and integrity of product by carrying out testing on different platforms • Identify errors and report required changes in software'

In [5]:
from dotenv import load_dotenv

load_dotenv()

True

In [7]:
from langchain.embeddings import HuggingFaceEmbeddings

model_name = "BAAI/bge-large-en-v1.5"
model_kwargs = {'device': 'cpu'}
embeddings = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
)

d:\multivector+opensource_llm\venv\Lib\site-packages\langchain_core\_api\deprecation.py:141: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(


In [8]:
import uuid
from langchain.schema import Document
from langchain_text_splitters import RecursiveCharacterTextSplitter

docs = [Document(page_content=text) for text in texts]
doc_ids = [str(uuid.uuid4()) for _ in docs]

id_key="doc_id"

# The splitter to use to create smaller chunks
child_text_splitter = RecursiveCharacterTextSplitter(chunk_size=8000, chunk_overlap=3000)
sub_docs = []
for i, doc in enumerate(docs):
    _id = doc_ids[i]
    _sub_docs = child_text_splitter.split_documents([doc])
    for _doc in _sub_docs:
        _doc.metadata[id_key] = _id
    sub_docs.extend(_sub_docs)

In [13]:
docs

[Document(page_content='WHAT IS COMPUTER SCIENCE ALL ABOUT?\n\nH. Conrad Cunningham and Pallavi Tadepalli Department of Computer and Information Science University of Mississippi\n\nCOMPUTERS EVERYWHERE As scientific and engineering disciplines go, computer science is still quite young. Although the mathematical roots of computer science go back more than a thousand years, it is only with the invention of the programmable electronic digital computer during the World War II era of the 1930s and 1940s that the modern discipline of computer science began to take shape. As it has developed, computer science includes theoretical studies, experimental methods, and engineering design all in one discipline.\n\nOne of the first computers was the ENIAC (Electronic Numerical Integrator and Computer), developed in the mid-1940s at the University of Pennsylvania. When construction was completed in 1946, the ENIAC cost about $500,000. In today’s terms, that is about $5,000,000. It weighed 30 tons, o

In [9]:
sub_docs

[Document(metadata={'doc_id': '85bf81ea-9652-40c5-a6ae-9ce0d4a54d70'}, page_content='WHAT IS COMPUTER SCIENCE ALL ABOUT?\n\nH. Conrad Cunningham and Pallavi Tadepalli Department of Computer and Information Science University of Mississippi\n\nCOMPUTERS EVERYWHERE As scientific and engineering disciplines go, computer science is still quite young. Although the mathematical roots of computer science go back more than a thousand years, it is only with the invention of the programmable electronic digital computer during the World War II era of the 1930s and 1940s that the modern discipline of computer science began to take shape. As it has developed, computer science includes theoretical studies, experimental methods, and engineering design all in one discipline.\n\nOne of the first computers was the ENIAC (Electronic Numerical Integrator and Computer), developed in the mid-1940s at the University of Pennsylvania. When construction was completed in 1946, the ENIAC cost about $500,000. In t

In [19]:
CONTEXTUAL_EMBEDDING_PROMPT = """
Here is the chunk we want to situate within the whole document:

{chunk}

Here is the content of the whole document:

{document}

Please provide a short, succinct context to situate this chunk within the overall document to improve search retrieval. Respond only with the context.
"""

In [11]:
from langchain_ollama import ChatOllama

def get_llm(model:str):
    model = ChatOllama(
                    model=model,
                    num_gpu=-1,
                    num_ctx=512,
                    num_thread=4,  # Adjust based on your CPU
                    num_predict=64,
                    f16_kv=True)
    return model

In [38]:
from typing import List
from langchain.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser
from langchain.chains import LLMChain

def create_contextual_chunks(chunks: List[Document], document: str) -> List[str]:
    contextual_chunks = []
    
    prompt_template = ChatPromptTemplate.from_template(CONTEXTUAL_EMBEDDING_PROMPT)
    # prompt = prompt_template.format(chunk=chunk, document=document)
    
    llm = get_llm(model='llama3.2:1b')

    #     chunk_context=prompt | llm | StrOutputParser()
        
    #     contextual_chunks.append(f"{chunk}\n{chunk_context}")
    # return contextual_chunks
    chain = LLMChain(llm=llm, prompt=prompt_template, output_parser=StrOutputParser())
    
    for chunk in chunks:
        # Extract the content from the chunk
        chunk_content = chunk.page_content
        
        # Run the chain
        chunk_context = chain.run(chunk=chunk_content, document=document)
        
        contextual_chunks.append(f"{chunk_content}\n{chunk_context}")
    
    print(contextual_chunks)
    return contextual_chunks

In [39]:
create_contextual_chunks(chunks=sub_docs,document=docs)

['WHAT IS COMPUTER SCIENCE ALL ABOUT?\n\nH. Conrad Cunningham and Pallavi Tadepalli Department of Computer and Information Science University of Mississippi\n\nCOMPUTERS EVERYWHERE As scientific and engineering disciplines go, computer science is still quite young. Although the mathematical roots of computer science go back more than a thousand years, it is only with the invention of the programmable electronic digital computer during the World War II era of the 1930s and 1940s that the modern discipline of computer science began to take shape. As it has developed, computer science includes theoretical studies, experimental methods, and engineering design all in one discipline.\n\nOne of the first computers was the ENIAC (Electronic Numerical Integrator and Computer), developed in the mid-1940s at the University of Pennsylvania. When construction was completed in 1946, the ENIAC cost about $500,000. In today’s terms, that is about $5,000,000. It weighed 30 tons, occupied as much space 

['WHAT IS COMPUTER SCIENCE ALL ABOUT?\n\nH. Conrad Cunningham and Pallavi Tadepalli Department of Computer and Information Science University of Mississippi\n\nCOMPUTERS EVERYWHERE As scientific and engineering disciplines go, computer science is still quite young. Although the mathematical roots of computer science go back more than a thousand years, it is only with the invention of the programmable electronic digital computer during the World War II era of the 1930s and 1940s that the modern discipline of computer science began to take shape. As it has developed, computer science includes theoretical studies, experimental methods, and engineering design all in one discipline.\n\nOne of the first computers was the ENIAC (Electronic Numerical Integrator and Computer), developed in the mid-1940s at the University of Pennsylvania. When construction was completed in 1946, the ENIAC cost about $500,000. In today’s terms, that is about $5,000,000. It weighed 30 tons, occupied as much space 

In [ ]:
import os
from langchain_qdrant import QdrantVectorStore

vectorstore = QdrantVectorStore.from_documents(
    documents=sub_docs,
    embedding=embeddings,
    collection_name="doc_stores_new",
    url=os.environ.get('QDRANT_URL'), 
    api_key=os.environ.get('QDRANT_API_KEY'),
)